# Xlsum Summarization

1. Import dependencies 

In [1]:
import random
import torch
import numpy as np
import pandas as pd
import wandb

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    pipeline,
)

from src.util.eval import evaluate_summery
from src.util.torch_device import resolve_torch_device
from src.data.xlsum import load_xlsum
from src.metrics.summarization import compute_metrics
from src.definitions import MODELS_FOLDER

2. Init WANB

In [2]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alexander-melashchenko (alexander-melashchenko-igor-sikorsky-kyiv-polytechnic-in) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

2. Config

In [3]:
random_seed = 42

random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)

device = resolve_torch_device()

In [4]:
model_checkpoint = "facebook/bart-large"
dataset_name = "csebuetnlp/xlsum"
language = "english"
batch_size = 4
num_train_epochs = 3
max_input_length = 512
max_target_length = 64
train_size = 0.8

In [5]:
run_name = f"xlsum-{str(model_checkpoint).split("/")[-1]}"
output_dir = MODELS_FOLDER / f"{run_name}-checkpoint"
resume_from_checkpoint = None

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

3. Load dataset

In [7]:
ds = load_xlsum(tokenizer, max_input_length, max_target_length, language)

In [8]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 306522
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11535
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11535
    })
})

4. Train

In [9]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    bf16=True,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=100,
    report_to="wandb",
    run_name=run_name,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics(tokenizer),
)

/home/melal/Workspace/nlp/.venv/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
torch.cuda.empty_cache()

In [ ]:
trainer.train(resume_from_checkpoint=resume_from_checkpoint)

In [11]:
wandb.finish()

5. Save weights

In [ ]:
trainer.save_model(MODELS_FOLDER / run_name)
tokenizer.save_pretrained(MODELS_FOLDER / run_name)

6. Test model

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODELS_FOLDER / run_name)
tokenizer = AutoTokenizer.from_pretrained(MODELS_FOLDER / run_name)

In [13]:
summarizer = pipeline(
    "summarization", model=model, tokenizer=tokenizer, truncation=True
)

Device set to use cuda:0


In [14]:
tuned_results = evaluate_summery(summarizer, ds, max_target_length, model_checkpoint)

f"Eval result is {tuned_results}"

100%|██████████| 11535/11535 [51:12<00:00,  3.75it/s] 


"Eval result is {'rouge1': np.float64(0.42241299994687187), 'rouge2': np.float64(0.1997336748109491), 'rougeL': np.float64(0.34443485056882484), 'rougeLsum': np.float64(0.34429214309547196), 'precision': np.float64(0.7519556778253391), 'recall': np.float64(0.7309160357178885), 'f1': np.float64(0.7402467150707807)}"

7. Test default model

In [15]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [16]:
summarizer = pipeline(
    "summarization", model=model, tokenizer=tokenizer, truncation=True
)

Device set to use cuda:0


In [17]:
default_results = evaluate_summery(summarizer, ds, max_target_length, model_checkpoint)

f"Eval result is {default_results}"

100%|██████████| 11535/11535 [1:32:45<00:00,  2.07it/s] 


"Eval result is {'rouge1': np.float64(0.19163649934032834), 'rouge2': np.float64(0.028179241294953715), 'rougeL': np.float64(0.12666462023334307), 'rougeLsum': np.float64(0.12666846740620316), 'precision': np.float64(0.5175382759258972), 'recall': np.float64(0.6168560410300322), 'f1': np.float64(0.5623211802498934)}"

8. Сomparison table

In [18]:
tuned_results["eval_name"] = "Tuned model"
default_results["eval_name"] = "Default model"

df = pd.DataFrame([default_results, tuned_results])

column_order = [
    "eval_name",
    "rouge1",
    "rouge2",
    "rougeL",
    "rougeLsum",
    "precision",
    "recall",
    "f1",
]

df = df[column_order]

df.head()

,eval_name,rouge1,rouge2,rougeL,rougeLsum,precision,recall,f1
0,Default model,0.191636,0.028179,0.126665,0.126668,0.517538,0.616856,0.562321
1,Tuned model,0.422413,0.199734,0.344435,0.344292,0.751956,0.730916,0.740247
